# Punto 4

## Librerías Utilizadas

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor

Iniciamos cargando el conjunto de datos

In [3]:
wind = pd.read_csv("https://raw.githubusercontent.com/lihkir/Data/main/wind_speed/data_treino_dv_df_2000_2010.csv")
wind.head()

,HORA (UTC),"VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))","VENTO, VELOCIDADE HORARIA (m/s)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C),TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C),"UMIDADE RELATIVA DO AR, HORARIA (%)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)","PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)","VENTO, RAJADA MAXIMA (m/s)",PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,886.5


Luego, corregimos los nombres de las columnas para facilitar su manipulación

In [4]:
columnas = {'HORA (UTC)': 'hora', 'VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))': 'direccion_viento', 
            'VENTO, VELOCIDADE HORARIA (m/s)': 'velocidad_viento', 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'humedad_max',
            'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'humedad_min', 'TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C)': 'temperatura_max',
            'TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C)': 'temperatura_min', 'UMIDADE RELATIVA DO AR, HORARIA (%)': 'humedad_horaria',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pres_atmosferica', 'PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)': 'precipitacion_hora',
            'VENTO, RAJADA MAXIMA (m/s)': 'rafaga_max', 'PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'pres_atmosferica_max', 'PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'pres_atmosferica_min'}
wind = wind.rename(columns=columnas)
wind.head()

,hora,direccion_viento,velocidad_viento,humedad_max,humedad_min,temperatura_max,temperatura_min,humedad_horaria,pres_atmosferica,precipitacion_hora,rafaga_max,pres_atmosferica_max,pres_atmosferica_min
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,886.5


Eliminamos la variable temporal

In [5]:
wind = wind.drop(columns=['hora'])

Iniciemos calculando los tamaños de las particiones. En este caso se quieren realizar 5 particiones del total de datos, por lo que dividimos el número de filas del dataset entre 5. Luego, hallamos el tamaño del conjunto de entrenamiento de cada partición, en este ejercicio se tomó un 80% de datos de entremamiento, por lo que el 20% restante es de prueba.

In [6]:
paso = len(wind) // 5
tamaño_entrenamiento = int(paso * 0.8)
tamaño_prueba = paso - tamaño_entrenamiento

Ahora realizamos una función que será la encargada de entrenar cada modelo (KNN Regressor, Linear Regression, Ridge y Lasso) y hallar las métricas de interés de cada modelo. Posteriormente se guardan en un diccionario y se retorna al algoritmo principal.

In [ ]:
def entrenar_y_evaluar_modelos(X_train, y_train, X_test, y_test):
    modelos = {
        "K-NN": KNeighborsRegressor(),
        "Linear Regression": LinearRegression(),
        "Ridge": Ridge(),
        "Lasso": Lasso()
    }
    
    metricas = {}
    
    for nombre_modelo, modelo in modelos.items():
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        metricas[nombre_modelo] = [mape, rmse, r2]
    
    return metricas

## Algoritmo Principal
Se inicia declarando el diccionario donde se guardaran los nombres de los modelos que se van a entrenar. Luego se encuentra el ciclo *for* que será el encargado de realizar la validación cruzada.
- En primer lugar nos aseguramos que en cada paso que haga el ciclo no exceda el tamaño total del dataset. Guardamos los índices del conjunto de entrenamiento y prueba, y luego separamos el dataset con esos índices. 
- Llamamos a la función que creamos anteriormente y agregamos las métricas al diccionario. Iteramos sobre cada modelo y sus métricas acumuladas en el diccionario metricas. 
- Para cada modelo, calculamos el promedio de las métricas (MAPE, RMSE, R^2) a lo largo de todas las particiones de la validación cruzada. Por último construimos un dataframe con las métricas.

In [23]:
metricas = {modelo: [] for modelo in ["K-NN", "Linear Regression", "Ridge", "Lasso"]}

for i in range(0, len(wind), paso):
    
    if i + paso > len(wind):
        break
    
    indices_prueba = list(range(i, i + tamaño_prueba))
    indices_entrenamiento = [idx for idx in range(len(wind)) if idx not in indices_prueba]

    conjunto_entrenamiento = wind.iloc[indices_entrenamiento]
    conjunto_prueba = wind.iloc[indices_prueba]

    X_train = conjunto_entrenamiento.drop(columns=['velocidad_viento'])
    y_train = conjunto_entrenamiento['velocidad_viento']
    X_test = conjunto_prueba.drop(columns=['velocidad_viento'])
    y_test = conjunto_prueba['velocidad_viento']
    
    metricas = entrenar_y_evaluar_modelos(X_train, y_train, X_test, y_test)
    
    for modelo, metrica in metricas.items():
        metricas[modelo].append(metrica)

metricas_df = {'Modelo': [], 'MAPE': [], 'RMSE': [], 'R^2': []}

for modelo, metricas_modelo in metricas.items():
    promedio_metricas = np.mean(metricas_modelo, axis=0)
    metricas_df['Modelo'].append(modelo)
    metricas_df['MAPE'].append(promedio_metricas[0])
    metricas_df['RMSE'].append(promedio_metricas[1])
    metricas_df['R^2'].append(promedio_metricas[2])

df_metricas = pd.DataFrame(metricas_df)
print(df_metricas)


              Modelo          MAPE      RMSE       R^2
0               K-NN  8.362727e+13  0.692820  0.717423
1  Linear Regression  6.870638e+13  0.625855  0.768534
2              Ridge  6.870685e+13  0.625855  0.768534
3              Lasso  1.420004e+14  0.795377  0.628512


En primer lugar, se observa un MAPE extremadamente alto en todos los modelos, lo que puede indicar un sobreajuste de los datos. Sin embargo, el RMSE parece aceptable y el R^2 también. En resumen, no es claro si los modelos son confiables para predecir.